In [108]:
import pandas as pd
news=pd.read_csv("Train.csv")

In [109]:
df_test=pd.read_csv("Test.csv")

In [110]:
news

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [111]:
news['source']=news.groupby('Category').ngroup()

In [89]:
group=news.groupby(['source', 'Category']).size().reset_index()

In [90]:
group=group[['source','Category']]

In [91]:
group

,source,Category
0,0,business
1,1,entertainment
2,2,politics
3,3,sport
4,4,tech


In [92]:
y=news.source
x=news.drop('source',axis=1)

In [93]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
test=y_test.to_frame()
train=y_train.to_frame()
df_test=pd.merge(x_test, test, left_index=True, right_index=True)
df_train=pd.merge(x_train, train, left_index=True, right_index=True)

In [94]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stopWords = set(stopwords.words('english'))

def textcleaner_stem(Text): # Need Change here
    
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', Text.lower()) # Need Change here
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        if w not in stopWords:
            if len(w)>2:
                clean.append(ps.stem(w))
    return ' '.join(clean)

In [95]:
df_train['clean_text_stem'] = df_train.Text.apply(lambda x: textcleaner_stem(x))
df_test['clean_text_stem'] = df_test.Text.apply(lambda x: textcleaner_stem(x))

In [96]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stopWords = set(stopwords.words('english'))

def textcleaner_lemmas(Text): # Need Change here
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', Text.lower()) # Need Change here
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        if w not in stopWords:
            if len(w)>2:
                clean.append(lemmatizer.lemmatize(w))
    return ' '.join(clean)

In [97]:
df_train['clean_text_lemma'] = df_train.Text.apply(lambda x: textcleaner_lemmas(x))
df_test['clean_text_lemma'] = df_test.Text.apply(lambda x: textcleaner_lemmas(x))

In [98]:
df_train

,ArticleId,Text,Category,source,clean_text_stem,clean_text_lemma
1103,1108,blair stresses prosperity goals tony blair say...,politics,2,blair stress prosper goal toni blair say parti...,blair stress prosperity goal tony blair say pa...
85,1458,iranian misses israel match iranian striker va...,sport,3,iranian miss israel match iranian striker vahi...,iranian miss israel match iranian striker vahi...
1162,1913,elvis fans hold birthday bash elvis fans aroun...,entertainment,1,elvi fan hold birthday bash elvi fan around wo...,elvis fan hold birthday bash elvis fan around ...
614,11,berlin cheers for anti-nazi film a german movi...,entertainment,1,berlin cheer anti nazi film german movi anti n...,berlin cheer anti nazi film german movie anti ...
77,1055,vodafone appoints new japan boss vodafone has ...,business,0,vodafon appoint new japan boss vodafon draft c...,vodafone appoints new japan bos vodafone draft...
...,...,...,...,...,...,...
940,253,ukip candidate suspended in probe eurosceptic ...,politics,2,ukip candid suspend probe euroscept parti ukip...,ukip candidate suspended probe eurosceptic par...
922,4,yeading face newcastle in fa cup premiership s...,sport,3,yead face newcastl cup premiership side newcas...,yeading face newcastle cup premiership side ne...
876,1202,parmalat sues 45 banks over crash parmalat has...,business,0,parmalat sue bank crash parmalat su bank tri r...,parmalat sue bank crash parmalat sued bank try...
726,1114,satellite mapping aids darfur relief aid worke...,tech,4,satellit map aid darfur relief aid worker tri ...,satellite mapping aid darfur relief aid worker...


In [99]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=6, strip_accents='ascii', analyzer='word', lowercase=True,
                             ngram_range=(1,2))

x_train_stem = vectorizer.fit_transform(df_train['clean_text_stem'])
y_train_stem = df_train['source']
x_test_stem = vectorizer.transform(df_test['clean_text_stem'])
y_test_stem = df_test['source']
features_train = vectorizer.get_feature_names()

In [100]:
from sklearn.naive_bayes import MultinomialNB

#Initialize and fit
nb = MultinomialNB()
nb.fit(x_train_stem, y_train_stem)

# Apply to testing data
y_pred_stem = nb.predict(x_test_stem)


# Showing model performance
print("Accuracy is: %0.3f" % nb.score(x_test_stem, y_test_stem))

Accuracy is: 0.966


In [101]:
vectorizer = TfidfVectorizer(min_df=6, strip_accents='ascii', analyzer='word', lowercase=True,
                             ngram_range=(1,2))

x_train_lemma = vectorizer.fit_transform(df_train['clean_text_lemma'])
y_train_lemma = df_train['source']
x_test_lemma = vectorizer.transform(df_test['clean_text_lemma'])
y_test_lemma = df_test['source']
features_train = vectorizer.get_feature_names()
len(features_train)

6376

In [102]:
from sklearn.naive_bayes import MultinomialNB


#Initialize and fit
nb = MultinomialNB()
nb.fit(x_train_lemma, y_train_lemma)

# Apply to testing data
y_pred_lemma = nb.predict(x_test_lemma)


# Showing model performance
print("Accuracy is: %0.3f" % nb.score(x_test_lemma, y_test_lemma))

Accuracy is: 0.966


In [103]:
X_train =  x_train_lemma
Y_train = y_train_lemma
X_test = x_test_lemma 
Y_test = y_test_lemma

In [104]:
from sklearn.svm import SVC


# Create instance and fit
sv = SVC(kernel='linear')
sv.fit(X_train, Y_train)

# Apply to testing data
y_pred = sv.predict(X_test)

# Showing model performance
cross = pd.crosstab(y_pred, Y_test)
print("Accuracy is: %0.3f" % sv.score(X_test, Y_test))

Accuracy is: 0.970


In [105]:
df_test['predict']=y_pred

In [107]:
df_test

,ArticleId,Text,Category,source,clean_text_stem,clean_text_lemma,predict
1132,1113,moya clinches cup for spain spain won the davi...,sport,3,moya clinch cup spain spain davi cup second ti...,moya clinch cup spain spain davis cup second t...,3
669,730,mansfield 0-1 leyton orient an second-half goa...,sport,3,mansfield leyton orient second half goal andi ...,mansfield leyton orient second half goal andy ...,3
408,1483,youssou n dour wins music prize senegalese mus...,entertainment,1,youssou dour win music prize senegales musicia...,youssou dour win music prize senegalese musici...,1
20,1547,hewitt overcomes wobble in sydney lleyton hewi...,sport,3,hewitt overcom wobbl sydney lleyton hewitt gav...,hewitt overcomes wobble sydney lleyton hewitt ...,3
613,2054,domain system opens door to scams a system to ...,tech,4,domain system open door scam system make easie...,domain system open door scam system make easie...,4
...,...,...,...,...,...,...,...
927,204,how to make a gigapixel picture the largest di...,tech,4,make gigapixel pictur largest digit panoram ph...,make gigapixel picture largest digital panoram...,4
1284,934,mutant book wins guardian prize a book about t...,entertainment,1,mutant book win guardian prize book evolut mut...,mutant book win guardian prize book evolution ...,1
1139,213,chepkemei joins edinburgh line-up susan chepke...,sport,3,chepkemei join edinburgh line susan chepkemei ...,chepkemei join edinburgh line susan chepkemei ...,3
1111,2015,desailly backs blues revenge trip marcel desai...,sport,3,desailli back blue reveng trip marcel desailli...,desailly back blue revenge trip marcel desaill...,3


In [78]:
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression()
lr.fit(X_train, Y_train)

y_pred = lr.predict(X_test)


# Showing model performance
cross = pd.crosstab(y_pred, Y_test)
print("Accuracy is: %0.3f" % lr.score(X_test, Y_test))

Accuracy is: 0.966


In [79]:
from sklearn import ensemble


gbc = ensemble.GradientBoostingClassifier()
gbc.fit(X_train, Y_train)

pred = gbc.predict(X_test)

# Showing model performance
cross = pd.crosstab(y_pred, Y_test)
print("Accuracy is: %0.3f" % gbc.score(X_test, Y_test))

Accuracy is: 0.940


In [80]:
from sklearn import ensemble


rfc = ensemble.RandomForestClassifier()
rfc.fit(X_train, Y_train)

y_pred = rfc.predict(X_test)


# Showing model performance
cross = pd.crosstab(y_pred, Y_test)
print("Accuracy is: %0.3f" % rfc.score(X_test, Y_test))

Accuracy is: 0.960
